## Data Cleaning For Stats Data

This project contains data of players that was scraped by me from fbref.com. The goal of this project is to clean the data so it can be used to create a KMeans clustering algorithim. One of the challenges faced is that players who have been used in more than one position in matches have two positions recorded in the data. The first approach was to take only the first position for players with two positions but that would have led to some of the players position being in accurate. To solve this problem it was neccessary to go back and scrape the squad data for the leagues present in the original data set and use the position from the squad data to update the position in the original dataset. The second problem with cleaning the data was that the player names in the original dataset and the squad dataset were inconsistent so the names could not be used as a unique identifier to fix the player position problem. The last resort was to use `scikit-learn` text extraction function `TfidfVectorizer` and then use `cosine_similarity` to compare how similar two player names are and update the position with the position of the player with the most similar name.

In [1]:
#importing required libraries
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
pd.set_option("display.max_columns", None)

In [3]:
defense = pd.read_csv("defense.csv")
passing = pd.read_csv("passing.csv")
shooting = pd.read_csv("shooting.csv")
goalkeeping = pd.read_csv("goalkeeping.csv")
gca = pd.read_csv("gca.csv")

In [4]:
gca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2889 entries, 0 to 2888
Data columns (total 26 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   (Unnamed: 0_level_0, Rk)        2889 non-null   int64  
 1   (Unnamed: 1_level_0, Player)    2889 non-null   object 
 2   (Unnamed: 2_level_0, Nation)    2888 non-null   object 
 3   (Unnamed: 3_level_0, Pos)       2889 non-null   object 
 4   (Unnamed: 4_level_0, Squad)     2889 non-null   object 
 5   (Unnamed: 5_level_0, Comp)      2889 non-null   object 
 6   (Unnamed: 6_level_0, Age)       2889 non-null   int64  
 7   (Unnamed: 7_level_0, Born)      2889 non-null   int64  
 8   (Unnamed: 8_level_0, 90s)       2889 non-null   float64
 9   (SCA, SCA)                      2885 non-null   float64
 10  (SCA, SCA90)                    2885 non-null   float64
 11  (SCA Types, PassLive)           2885 non-null   float64
 12  (SCA Types, PassDead)           28

In [5]:
shooting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2889 entries, 0 to 2888
Data columns (total 27 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   (Unnamed: 0_level_0, Rk)        2889 non-null   int64  
 1   (Unnamed: 1_level_0, Player)    2889 non-null   object 
 2   (Unnamed: 2_level_0, Nation)    2888 non-null   object 
 3   (Unnamed: 3_level_0, Pos)       2889 non-null   object 
 4   (Unnamed: 4_level_0, Squad)     2889 non-null   object 
 5   (Unnamed: 5_level_0, Comp)      2889 non-null   object 
 6   (Unnamed: 6_level_0, Age)       2889 non-null   int64  
 7   (Unnamed: 7_level_0, Born)      2889 non-null   int64  
 8   (Unnamed: 8_level_0, 90s)       2889 non-null   float64
 9   (Standard, Gls)                 2889 non-null   int64  
 10  (Standard, Sh)                  2889 non-null   int64  
 11  (Standard, SoT)                 2889 non-null   int64  
 12  (Standard, SoT%)                23

In [6]:
passing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2889 entries, 0 to 2888
Data columns (total 33 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   (Unnamed: 0_level_0, Rk)        2889 non-null   int64  
 1   (Unnamed: 1_level_0, Player)    2889 non-null   object 
 2   (Unnamed: 2_level_0, Nation)    2888 non-null   object 
 3   (Unnamed: 3_level_0, Pos)       2889 non-null   object 
 4   (Unnamed: 4_level_0, Squad)     2889 non-null   object 
 5   (Unnamed: 5_level_0, Comp)      2889 non-null   object 
 6   (Unnamed: 6_level_0, Age)       2889 non-null   int64  
 7   (Unnamed: 7_level_0, Born)      2889 non-null   int64  
 8   (Unnamed: 8_level_0, 90s)       2889 non-null   float64
 9   (Total, Cmp)                    2885 non-null   float64
 10  (Total, Att)                    2885 non-null   float64
 11  (Total, Cmp%)                   2861 non-null   float64
 12  (Total, TotDist)                28

In [7]:
goalkeeping.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 28 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   (Unnamed: 0_level_0, Rk)        208 non-null    int64  
 1   (Unnamed: 1_level_0, Player)    208 non-null    object 
 2   (Unnamed: 2_level_0, Nation)    208 non-null    object 
 3   (Unnamed: 3_level_0, Pos)       208 non-null    object 
 4   (Unnamed: 4_level_0, Squad)     208 non-null    object 
 5   (Unnamed: 5_level_0, Comp)      208 non-null    object 
 6   (Unnamed: 6_level_0, Age)       208 non-null    int64  
 7   (Unnamed: 7_level_0, Born)      208 non-null    int64  
 8   (Playing Time, MP)              208 non-null    int64  
 9   (Playing Time, Starts)          208 non-null    int64  
 10  (Playing Time, Min)             208 non-null    int64  
 11  (Unnamed: 11_level_0, 90s)      208 non-null    float64
 12  (Performance, GA)               207 

In [8]:
defense.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2889 entries, 0 to 2888
Data columns (total 26 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   (Unnamed: 0_level_0, Rk)        2889 non-null   int64  
 1   (Unnamed: 1_level_0, Player)    2889 non-null   object 
 2   (Unnamed: 2_level_0, Nation)    2888 non-null   object 
 3   (Unnamed: 3_level_0, Pos)       2889 non-null   object 
 4   (Unnamed: 4_level_0, Squad)     2889 non-null   object 
 5   (Unnamed: 5_level_0, Comp)      2889 non-null   object 
 6   (Unnamed: 6_level_0, Age)       2889 non-null   int64  
 7   (Unnamed: 7_level_0, Born)      2889 non-null   int64  
 8   (Unnamed: 8_level_0, 90s)       2889 non-null   float64
 9   (Tackles, Tkl)                  2885 non-null   float64
 10  (Tackles, TklW)                 2889 non-null   int64  
 11  (Tackles, Def 3rd)              2885 non-null   float64
 12  (Tackles, Mid 3rd)              28

All of the columns are multi-index columns. Some of the column names in the second level of the columns have duplicate names and so the column name on the first level give meaning to those columns. To fix the column names, a combination of both the first and second level column name will be done for columns with duplicates in the second level and then the first level will be dropped.

In [9]:
passing.columns = [col[1].lower() if "Unnamed" in col[0]
                  else f"{col[0]}_{col[1]}".lower().replace(" ", "_")
                  for col in passing.columns]

gca.columns = [col[1].lower() if "Unnamed" in col[0] or "SCA" in col[1] or "GCA" in col[1]
               else f"{col[0]}_{col[1]}".lower().replace("types_", "").replace(" ", "_")
               for col in gca.columns]


defense.columns = [col[1].lower() if "Unnamed" in col[0] or "Blocks" in col[1]
                  else f"{col[0]}_{col[1]}".lower().replace(" ", "_")
                  for col in defense.columns]

shooting.columns = shooting.columns.droplevel().str.lower()
goalkeeping.columns = [col[1].replace("Save%", "pk_save%") if "Penalty Kicks" in col[0]
                        and "Save%" in col[1] else col[1].lower() for col in goalkeeping.columns]

To clean the data, several functions that each perform a specific data cleaning task will be written. 

* The squad data have their positions spelled in full. This will be replaced with an abbreviation for the position to match that of the stats data. E.g `Defender` will become `DF`. This will be done by the first fucntion.
* The next two functions `get_player_position` and `pos_clean` is used to query the squad data and use the player position from the squad data to replace the position of players who have more than 1 position in the stats data.
* The last two functions `clean` and `save_data` are used to do the final data cleaning and to save the dataframe respectfully. 

In [10]:
pl_squads = pd.read_parquet("pl_squads.csv")
la_liga_squads = pd.read_parquet("laliga_squads.csv")
ligue1_squads = pd.read_parquet("ligue1_squads.csv")
bundesliga_squads = pd.read_parquet("bundesliga_squads.csv")

# the column name of the bundesliga dataframe isnt't consistent with the rest
# It had to be corrected using one the column names from the previous dataframes
serie_a_squads = pd.read_parquet("serie_a_squads.parquet")

bundesliga_squads.columns = pl_squads.columns 

top5_leagues = pd.concat([pl_squads, la_liga_squads, ligue1_squads, bundesliga_squads, serie_a_squads])
top5_leagues.reset_index(drop=True, inplace=True)

In [11]:
def clean_squad_positions(df_: pd.DataFrame) -> pd.DataFrame:
    return (df_.assign(position = df_["position"].apply(
        lambda x: "GK" if x == "Goalkeeper" else "DF" if x == "Defender"
                    else "MF" if x == "Midfielder" else "FW")
                      )
           )

In [12]:
top5_leagues = clean_squad_positions(top5_leagues)

In [13]:
def get_player_position(player_name: str) -> str:
    """ Function to get the correct player position"""
    vectorizer = TfidfVectorizer(ngram_range = (1,2)) # using both a unigram and bigram to match on when either one or two of the names match
    tfidf = vectorizer.fit_transform(top5_leagues["player_name"])
    search_vector = vectorizer.transform([player_name])
    similarity = cosine_similarity(search_vector, tfidf).flatten()
    index = np.where(similarity == similarity.max()) # returns the index of the player name that matches the most
    position = top5_leagues.loc[index]["position"].values[0]
    
    return position

In [14]:
def pos_clean(df_: pd.DataFrame) -> pd.DataFrame:
    double_players = df_.loc[df_["pos"].apply(len) > 2].copy()
    double_players_index = double_players.index
    df_.loc[double_players_index, "pos"] = double_players["player"].apply(get_player_position)
    
    return df_
    

In [15]:
def clean(df_: pd.DataFrame) -> pd.DataFrame:
    """Function to clean the stats dataframe"""
    
    return (df_.drop(["rk", "matches"], axis=1)
            .assign(nation = gca["nation"].str.extract(r"([A-Z]+)").fillna("ENG"),
                    comp = df_["comp"].apply(lambda x: " ".join(x.split()[1:]))
                    )
            .pipe(pos_clean)
            .fillna(0.0)
           )


In [16]:
def save_data() -> None:
    dfs = [defense, passing, shooting, goalkeeping, gca]
    names = ["defense", "passing", "shooting", "goalkeeping", "gca"]
    
    for df, name in zip(dfs, names):
        df = clean(df)
        df.to_parquet(f"{name}.csv", index=False)

In [17]:
save_data()